In [ ]:
import requests
import pandas as pd

In [ ]:
## Definining the API URLs an dfetching the data
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
pd.set_option('display.max_columns', None) 

bootstrap_response = requests.get(bootstrap_url)
boot_data = bootstrap_response.json()

fixtures_response = requests.get(fixtures_url)
fixtures_data = fixtures_response.json()

In [ ]:
# teams ---> A scomplete summary of the teams metrics
# elements ---> A complete summary of the players metrics
# element_stats ---> Has the metrics we are interested in for each player
# element_types ---> Has the player positions (Goalkeeper, Defender, Midfielder, Forward)

In [ ]:
display(pd.DataFrame(boot_data['element_stats']))

In [ ]:
display(pd.DataFrame(boot_data['element_types']))

In [ ]:
# Analyzing the player dtaa from the 'elements' key
player_data = pd.DataFrame(boot_data['elements'])
player_data.head()

In [ ]:
# De-cluttering the players dataframe to only the columns we are interested in
player_df_cols = [i['name'] for i in boot_data['element_stats']]
players_df = player_data[['web_name', 'team', 'element_type', 'now_cost', 'selected_by_percent', 'total_points'] + player_df_cols].copy()
players_df.head()

In [ ]:
# Teams data from the 'teams' key
team_data = pd.DataFrame(boot_data['teams'])

In [ ]:
# Mapping team IDs to team names and positions in the de-cluttered players dataframe
team_map = {team['id']: team['name'] for team in boot_data['teams']}
players_df['team'] = players_df['team'].map(team_map)

positions_map = {pos['id']: pos['singular_name'] for pos in boot_data['element_types']}
players_df['element_type'] = players_df['element_type'].map(positions_map)
players_df.head()

In [ ]:
players_df.info(verbose=True, show_counts=True)

In [ ]:
pd.DataFrame(fixtures_data['stats'])

In [ ]:
# Diving dinto fixtures

fixtures_df = pd.DataFrame(fixtures_data)
fixtures_df.head()

In [ ]:
fixtures_df.info(verbose=True, show_counts=True)

In [ ]:
cleaned_fixtures_df = fixtures_df[['event', 'team_h', 'team_a', 'team_h_score', 'team_a_score', 'team_h_difficulty', 'team_a_difficulty', 'finished']].copy()
cleaned_fixtures_df['team_h'] = cleaned_fixtures_df['team_h'].map(team_map)
cleaned_fixtures_df['team_a'] = cleaned_fixtures_df['team_a'].map(team_map)
cleaned_fixtures_df.head()

In [ ]:
historical_fixtures = cleaned_fixtures_df[cleaned_fixtures_df['finished'] == True].copy()
future_fixtures = cleaned_fixtures_df[cleaned_fixtures_df['finished'] == False].copy() 

In [ ]:
# 1. Calculate the League Average for the current season
# This automatically updates every time you run your script with new data
league_average_conceded = df['goals_conceded'].mean()

# 2. Calculate the specific Team's Rolling Average (Last 3 games)
team_rolling_avg = df.groupby('team_id')['goals_conceded'].transform(lambda x: x.rolling(3).mean())

# 3. Create the Dynamic Form Multiplier
# If the team concedes 3.0 and the league average is 1.2, multiplier = 2.5
df['defensive_form_multiplier'] = team_rolling_avg / league_average_conceded